In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

def get_resnet18():
    model = models.resnet18(pretrained=True)
    return model

def get_vgg16():
    model = models.vgg16(pretrained=True,weights=VGG16_Weights.IMAGENET1K_V1)
    return model

def get_inception_v3():
    model = models.inception_v3(pretrained=True,weights=Inception_V3_Weights.IMAGENET1K_V1)
    return model

def get_mobilenet_v2():
    model = models.mobilenet_v2(pretrained=True,weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    return model

def get_densenet121():
    model = models.densenet121(pretrained=True,weights=DenseNet121_Weights.IMAGENET1K_V1)
    return model
    
def extract_features(model, input_image_path):
    input_image = Image.open(input_image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_batch = input_batch.cuda()
        model.eval().cuda()

    with torch.no_grad():
        output = model(input_batch)
    return output[0]


In [ ]:
model = get_resnet18()
output = extract_features(model,"/kaggle/input/fortest/pexels-pixabay-45201.jpg")
print(output)

In [ ]:
model = get_vgg16()
output1 = extract_features(model,"/kaggle/input/fortest/pexels-pixabay-45201.jpg")
print(output1)

In [ ]:
model = get_inception_v3()
output2 = extract_features(model,"/kaggle/input/fortest/pexels-pixabay-45201.jpg")
print(output2)

In [ ]:
model = get_mobilenet_v2()
output3 = extract_features(model,"/kaggle/input/fortest/pexels-pixabay-45201.jpg")
print(output3)

In [ ]:
model = get_densenet121()
output4 = extract_features(model,"/kaggle/input/fortest/pexels-pixabay-45201.jpg")
print(output4)

In [ ]:
# Import necessary libraries
import torch
from torchvision.models import detection
import torchvision
from PIL import Image
import numpy as np


def load_image(image_path):
    """
    Loads an image from the given file path using the PIL library.
    """
    image = Image.open(image_path)
    return image



# Define RetinaNet post processor
class RetinaNetPostProcessor(torch.nn.Module):
    def __init__(self, score_threshold=0.05, iou_threshold=0.5, max_detection_per_image=100):
        super(RetinaNetPostProcessor, self).__init__()
        self.score_threshold = score_threshold
        self.iou_threshold = iou_threshold
        self.max_detection_per_image = max_detection_per_image
        self.box_coder = detection.BoxCoder(weights=(1.0, 1.0, 1.0, 1.0))
        self.anchor_generator = detection.anchor_utils.AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
        self.box_similarity = detection.box_ops.box_iou
        self.proposal_matcher = detection.matcher.Matcher(self.iou_threshold, self.iou_threshold, allow_low_quality_matches=False)
        self.post_processor = detection.PostProcess(self.score_threshold, self.max_detection_per_image, self.box_coder, self.box_similarity, self.proposal_matcher)

    def forward(self, inputs, box_cls, box_regression):
        anchors = self.anchor_generator(inputs)
        detections = self.post_processor((box_cls,), (box_regression,), anchors)
        return detections[0]


# Define function to detect with RetinaNet
def detect_with_retinanet(image):
    # Load RetinaNet model in inference mode
    model = detection.retinanet_resnet50_fpn(pretrained=True, pretrained_backbone=True)
    model.eval()
    model.postprocess = RetinaNetPostProcessor()
    # Preprocess image
    image = torch.from_numpy(image).permute(2, 0, 1).float().unsqueeze(0)
    # Perform detection
    with torch.no_grad():
        detections = model(image)
    detections = model.postprocess(detections)
    return detections

# Define function to detect with FasterRCNN
def detect_with_fasterrcnn(image):
    # Load FasterRCNN model in inference mode
    model = detection.fasterrcnn_resnet50_fpn(pretrained=True, pretrained_backbone=True)
    model.eval()
    # Preprocess image
    image = torch.from_numpy(image).permute(2, 0, 1).float().unsqueeze(0)
    # Perform detection
    with torch.no_grad():
        detections = model(image)
    detections = model.postprocess(detections)
    return detections
    
# Define function to detect with SSD Lite
def detect_with_ssd_lite(image):
    # Load SSD Lite model in inference mode
    model = detection.ssd_lite_mobilenet_v3_large(pretrained=True, pretrained_backbone=True)
    model.eval()
    # Preprocess image
    image = torch.from_numpy(image).permute(2, 0, 1).float().unsqueeze(0)
    # Perform detection
    with torch.no_grad():
        detections = model(image)
    detections = model.postprocess(detections)
    return detections
    
# Define function to detect with Yolov3
def detect_with_yolov3(image):
    # Load Yolov3 model in inference mode
    model = torch.hub.load('ultralytics/yolov3', 'yolov3', pretrained=True)
    model.eval()
    # Preprocess image
    image = torch.from_numpy(image).permute(2, 0, 1).float().unsqueeze(0)
    # Perform detection
    with torch.no_grad():
        detections = model(image)
    detections = model.postprocess(detections)
    return detections

print(torchvision.__version__)
image = load_image("/kaggle/input/fortest/pexels-pixabay-45201.jpg")
# Create a list of tensors from the input data
array = np.array(image)

detect_with_retinanet(array)

In [ ]:
detect_with_fasterrcnn(array)

In [ ]:
detect_with_ssd_lite(array)

In [ ]:
detect_with_yolov3(array)